In [1]:
!pip install pickle5 pandas==1.3 wandb torchinfo torchviz
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchinfo import summary
from torchviz import make_dot
import wandb

from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

from random import randint
from google.colab import drive
import pickle5 as pickle
import warnings

from IPython.display import Image
from IPython.core.display import Image, display

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


In [2]:
# Load Data
def load_data():

  drive.mount('/content/drive')
  local_path = 'drive/My Drive/deep_learning/assignment/datasets/'
  #local_path = "C:/Users/simon/Nextcloud2/Master/Deep Learning/assignment/"
  data_path = local_path + "dataset_big_boi_unscaled.pickle"

  with open(data_path, "rb") as fh:
    dataset = pickle.load(fh)
  return dataset

In [3]:
def build_dataset(batch_size,max_iri_value):
  #if "dataset" not in globals():
  print("Loading Dataset")
  dataset = load_data()

  #data_shape_old = dataset["train"].shape
  #train_set = dataset["train"][["z","IRI_mean"]].drop(dataset["train"][["z","IRI_mean"]][dataset["train"]["IRI_mean"]>max_iri_value].index)
  train_set = dataset["train"][["z","IRI_mean"]][0:20]
  test_set = dataset["test"][["z","IRI_mean"]][0:20]
  #print(f"Shape of dataset: {data_shape_old}, after max iri: {train_set_l.shape}")
  #train_set_high_iri = dataset["train"][["z","IRI_mean"]].drop(dataset["train"][["z","IRI_mean"]][dataset["train"]["IRI_mean"]<=max_iri_value].index)
  #train_set = train_set_l["z"].values
  #test_set = dataset["test"]["z"].values
  #print(f"Dataset length reduced: {dataset['train'].shape[0]} -> {train_set.shape[0]}")
  return dataset, train_set, test_set

In [4]:
def build_model(device):
  class Model(nn.Module):
      def __init__(self):
          super(Model, self).__init__()
          self.linear1 = torch.nn.Linear(3085,600)
          self.linear2 = torch.nn.Linear(600,60)
          self.linear3 = torch.nn.Linear(60,2)
          self.relu = torch.nn.ReLU()   
          self.sm = torch.nn.Softmax()       
          
      def forward(self, x):
          x = self.linear1(x)
          x = self.linear2(self.relu(x))
          return self.sm(self.linear3(self.relu(x)))

  return Model().to(device)


In [5]:
def build_optimizer(model,learning_rate):
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
  loss_function = torch.nn.CrossEntropyLoss()
  return optimizer,loss_function

In [6]:
def train_epoch(epoch, model, train_set, test_set, optimizer, loss_function, device, batch_size,y_train,y_test):
  train_loss = 0
  test_loss = 0
  #rng = np.random.default_rng(epoch)
  #rng.shuffle(train_set,axis=0)
  #rng.shuffle(test_set, axis=0)

  model.train()
  for idx in range(train_set["z"].shape[0]//batch_size):
      batch = np.zeros((batch_size,1,train_set["z"].iloc[0].shape[0]))

      for b in range(batch_size):
        batch[b][0]=train_set["z"].values[idx*batch_size+b]

      batch = torch.from_numpy(batch.astype(np.float32)).to(device)
      output = model(batch)
      #print(f"Output shape: {output.shape}")
      #print(f"y_train shape: {y_train[idx*batch_size:idx*batch_size+batch_size].shape}")

      #print(f"output: {output}\n label: {y_train[idx*batch_size:idx*batch_size+batch_size].view(-1,1,1)}")
      loss = loss_function(output.reshape(-1,2), y_train[idx*batch_size:idx*batch_size+batch_size].reshape(-1,2))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()


  model.eval()
  with torch.no_grad():
    for idx in range(test_set["z"].shape[0]//batch_size):
        batch = np.zeros((batch_size,1,test_set["z"].iloc[0].shape[0]))
        for b in range(batch_size):
            batch[b][0]=test_set["z"].values[idx*batch_size+b]
        batch = torch.from_numpy(batch.astype(np.float32)).to(device)
        output = model(batch)
        loss = loss_function(output.reshape(-1,2), y_test[idx*batch_size:idx*batch_size+batch_size].reshape(-1,2))
        test_loss += loss.item()

  train_loss = train_loss/(train_set["z"].shape[0]//batch_size)
  test_loss = test_loss/(test_set["z"].shape[0]//batch_size)
  return train_loss, test_loss

In [ ]:
max_iri = 4
epochs = 20
batch_size=1

dataset, train_set, test_set=build_dataset(1,max_iri)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_model(device)
optimizer, loss_function = build_optimizer(model,2e-4)

y_train = np.array([0 if v<5 else 1 for v in train_set["IRI_mean"].values])
y_test = np.array([0 if v<5 else 1 for v in test_set["IRI_mean"].values])

y_train = torch.nn.functional.one_hot(torch.from_numpy(y_train)).to(device).float()
y_test = torch.nn.functional.one_hot(torch.from_numpy(y_test)).to(device).float()

X_train = dataset["train"]["z"].values
X_test = dataset["test"]["z"].values



for epoch in range(epochs):
  train_loss, test_loss = train_epoch(epoch, model,train_set,test_set,optimizer,loss_function,device,batch_size,y_train,y_test)
  
  print("Epoch: [{}/{}] Train-Loss: {:.3f}, Val-Loss: {:.3f}".format(epoch+1, 
                      epochs, train_loss, test_loss)) 